In [1]:
import nltk
import pandas as pd
import os

from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer

import pandas as pd
import re as regex
import numpy as np
import scipy
import os,glob
from sklearn.externals import joblib
import contractions

In [2]:
def contract(text):
    return contractions.fix(text)

def tokenize(text):
    words=word_tokenize(text)
    return words

def regTokenize(text):
    tok=RegexpTokenizer('[A-Za-z0-9]?\w+')
    return tok.tokenize(text)

def removePunct(text):
    words=wordpunct_tokenize(text)
    return words

def removePunctuations(words):
#     https://www.programiz.com/python-programming/examples/remove-punctuation
    punctuations='''!()-."[]".{};:'"\,<>./---::--...''``?@#$%^&*_~'''
    removed=[]
    for char in words:
        if char not in punctuations:
            removed.append(char)
        else:
            continue
    return removed
    
def lowercase(text):
    return text.lower()

def lemma(words):
    for i in range(0,len(words)):
        words[i]=WordNetLemmatizer().lemmatize(words[i])
    return words

def stemming(words):
    porter_stemmer=PorterStemmer()
    for i in range(0,len(words)):
        words[i]=porter_stemmer.stem(words[i])
    return words

def removestopwords(text):
    
    stopWords = set(stopwords.words('english'))
    wordsFiltered = []
    for w in text:
        if w not in stopWords:
            wordsFiltered.append(w)
    return wordsFiltered

def remove_numbers(words):
    pass

In [3]:
terms=[]
document_ids=[]

path = 'thirdSet'
print(path)
for dire in glob.glob(os.path.join(path, '*')):
    print(dire)
    for filename in glob.glob(os.path.join(dire, '*')):
        st=filename.split('\\')
#         print(st[2])
        s=st[1]+'/'
        s=s+st[2]
        
#         document_ids.append(st[2])
        document_ids.append(s)

        f=open(filename,'r+')
        data=f.read()

        data=lowercase(data)
        data=contract(data)
        data=regTokenize(data)
        data=lemma(data)
        data=stemming(data)

        data=list(data)

        for i in range(0,len(data)):
            t=data[i]
#             id=st[2]
            id=s
            row=[]
            
            row.append(t)
            row.append(id)
            row.append(i+1)
            terms.append(row)
    print('============================================================')

thirdSet
thirdSet\comp.graphics
thirdSet\rec.motorcycles


In [4]:
print(len(document_ids))
print(len(terms))

2000
633528


In [5]:
from operator import itemgetter
sorted_list = sorted(terms, key=itemgetter(0,1,2))

In [6]:
# for i in sorted_list:
#     print(i)

In [7]:
dictionary={}
for i in sorted_list:
    ind=i[0]
    docid=i[1]
    if(ind not in dictionary.keys()):
        dictionary[ind]=[]
        docfreq=1
        dictionary[ind].append(docfreq)
        
        doc_dict={}
        doc_dict[docid]=[i[2]]
        dictionary[ind].append(doc_dict)
    else:
        if(docid not in dictionary.get(ind)[1].keys()):
            dictionary.get(ind)[1][docid]=[i[2]]
        else:
            dictionary.get(ind)[1].get(docid).append(i[2])
            
        dictionary.get(ind)[0]=len(dictionary.get(ind)[1].keys())

In [8]:
documentIds_filename='document_ids_positional_index.sav'
joblib.dump(document_ids,documentIds_filename)

['document_ids_positional_index.sav']

In [9]:
terms_filename='terms_positional_index.sav'
joblib.dump(terms, terms_filename)

['terms_positional_index.sav']

In [10]:
sorted_list_filename='sorted_list_positional_index.sav'
joblib.dump(sorted_list,sorted_list_filename)

['sorted_list_positional_index.sav']

In [11]:
dictionary_filename='dictionary_positional_index.sav'
joblib.dump(dictionary,dictionary_filename)

['dictionary_positional_index.sav']

In [12]:
len(dictionary.keys())

26474

### Functions

In [13]:
di = joblib.load('dictionary_positional_index.sav')
docids=joblib.load('document_ids_positional_index.sav')

In [14]:
len(list(set(docids)))

2000

In [15]:
def pre_processing_test(term):
    term=lowercase(term)
    term=contract(term)
    term=regTokenize(term)
    term=lemma(term)
    term=stemming(term)
    return term

In [16]:
def search(query):
    print(query)
    distinct_words=set(query)
    distinct_words=list(distinct_words)
#     ---------------------------------------------
    q=[]#getting keys from positional index for each token in query
    for i in range(0,len(distinct_words)):
        q.append(di.get(distinct_words[i])[1].keys())
    
    z=0
    for i in q:
        print(distinct_words[z],'-->',len(i))
        z+=1
#     ---------------------------------------------
    keys_set=[] # converting list to set to perform intersection
    for i in range(0,len(q)):
        keys_set.append(set(q[i]))
    
#     for i in keys_set:
#         print(i)
#         print('\n')
#     ---------------------------------------------
    commom_docs_set=set.intersection(*keys_set)  # getting common_docs set => intersection of list of sets
#     print('\ncommon docs set:-->',commom_docs_set)
    
    common_docs_list=list(commom_docs_set)  # converting common docs set to list and sort
    common_docs_list.sort()
#     print('\ncommon docs list:-->',common_docs_list)
#     ---------------------------------------------
    searched_docs=[]
    for i in range(0,len(common_docs_list)):
        doc_list_position=[]
        doc_id=common_docs_list[i]
        for j in range(0,len(query)):
            term=query[j]
            doc_list_position.append(di.get(term)[1].get(doc_id))
            
#         for k in doc_list_position:
#             print(k)
        
#         print('==>',doc_list_position)
        for k in range(1,len(doc_list_position)):
            doc_list_position[k]=[x-k for x in doc_list_position[k]]
#         print('==>',doc_list_position)
        
        dc_list_set=[]
        for k in doc_list_position:
            dc_list_set.append(set(k))
#         print(dc_list_set)
        
        if(bool(set.intersection(*dc_list_set))!=0):
            searched_docs.append(doc_id)
            
    return searched_docs

In [17]:
x=str(input('enter query:'))

print(x)
x=pre_processing_test(x)
print(x)

retrieved_docs=search(x)
if(len(retrieved_docs)==0):
    print('No Documents have this Phrase.')
else:
    print('Documents with given Phrase:',retrieved_docs)
    print('\n',len(retrieved_docs))

enter query:algorithm to
algorithm to
['algorithm', 'to']
['algorithm', 'to']
to --> 1764
algorithm --> 108
Documents with given Phrase: ['comp.graphics/37949', 'comp.graphics/38238', 'comp.graphics/38246', 'comp.graphics/38248', 'comp.graphics/38249', 'comp.graphics/38267', 'comp.graphics/38355', 'comp.graphics/38375', 'comp.graphics/38637', 'comp.graphics/38734', 'comp.graphics/38852', 'comp.graphics/39049', 'comp.graphics/39060']

 13
